In [2]:
import numpy as np
import pandas as pd

# Define IAM role
import boto3
import re
import sagemaker
from sagemaker import get_execution_role

# SageMaker SDK Documentation: http://sagemaker.readthedocs.io/en/latest/estimators.html

## Upload Data to S3

In [ ]:
bucket_name = 'spk-sagemaker-ohio'
training_file_key = 'biketrain/bike_train.csv'
validation_file_key = 'biketrain/bike_validation.csv'
test_file_key = 'biketrain/bike_test.csv'

s3_model_output_location = r's3://{0}/biketrain/model'.format(bucket_name)
s3_training_file_location = r's3://{0}/{1}'.format(bucket_name,training_file_key)
s3_validation_file_location = r's3://{0}/{1}'.format(bucket_name,validation_file_key)
s3_test_file_location = r's3://{0}/{1}'.format(bucket_name,test_file_key)

In [ ]:
print(s3_model_output_location)
print(s3_training_file_location)
print(s3_validation_file_location)
print(s3_test_file_location)

In [ ]:
# http://boto3.readthedocs.io/en/latest/guide/s3.html
# S3 Naming conventions. 
# ----------------------
# files      = objects in S3.  
# file name  = key name in S3.

def write_to_s3(filename, bucket, key):
    with open(filename,'rb') as f: # Read in binary mode
        return boto3.Session().resource('s3').Bucket(bucket).Object(key).upload_fileobj(f)

In [ ]:
write_to_s3('bike_train.csv',bucket_name,training_file_key)
write_to_s3('bike_validation.csv',bucket_name,validation_file_key)
write_to_s3('bike_test.csv',bucket_name,test_file_key)

## Training Algorithm Docker Image
### AWS Maintains a separate image for every region and algorithm

In [ ]:
# Registry Path for algorithms provided by SageMaker
#  https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-algo-docker-registry-paths.html

containers = {'us-west-2': '433757028032.dkr.ecr.us-west-2.amazonaws.com/xgboost:latest',  # Oregon 
              'us-east-1': '811284229777.dkr.ecr.us-east-1.amazonaws.com/xgboost:latest',  # N.Virginia
              'us-east-2': '825641698319.dkr.ecr.us-east-2.amazonaws.com/xgboost:latest',  # Ohio
              'eu-west-1': '685385470294.dkr.ecr.eu-west-1.amazonaws.com/xgboost:latest'}  # Ireland 

In [ ]:
role = get_execution_role()

In [ ]:
# This role contains the permissions needed to train, deploy models
# SageMaker Service is trusted to assume this role
print(role)

## Build Model

In [ ]:
sess = sagemaker.Session()

In [ ]:
# Access appropriate algorithm container image
#  Specify how many instances to use for distributed training and what type of machine to use
#  Finally, specify where the trained model artifacts needs to be stored
#   Reference: http://sagemaker.readthedocs.io/en/latest/estimators.html
#    Optionally, give a name to the training job using base_job_name

# https://docs.aws.amazon.com/sagemaker/latest/dg/ex1-train-model.html

estimator = sagemaker.estimator.Estimator(containers[boto3.Session().region_name],
                                       role, 
                                       train_instance_count=1, 
                                       train_instance_type='ml.m4.xlarge',
                                       output_path=s3_model_output_location,
                                       sagemaker_session=sess,
                                       base_job_name ='xgboost-biketrain-v1')

In [ ]:
#spk ##
xgb_model = sagemaker.estimator.Estimator(containers,
                                         role, 
                                         train_instance_count=1, 
                                         train_instance_type='ml.m4.xlarge',
                                         train_volume_size = 5,
                                         output_path=s3_model_output_location,
                                         sagemaker_session=sagemaker.Session())

In [ ]:
#spk ##

xgb_model.set_hyperparameters(max_depth = 5,
                              eta = .1,
                              gamma = 4,
                              min_child_weight = 6,
                              silent = 0,
                              objective = "reg:linear",
                              num_class = 10,
                              num_round = 150,
                              subsample = 0.7
                             )

In [ ]:
# Specify hyper parameters that appropriate for the training algorithm
# XGBoost Training Parameter Reference: 
#   https://github.com/dmlc/xgboost/blob/master/doc/parameter.rst

# max_depth=5,eta=0.1,subsample=0.7,num_round=150
estimator.set_hyperparameters(max_depth=5,objective="reg:linear",
                              eta=0.1,subsample=0.7,num_round=150)

In [ ]:
xgb_model.hyperparameters()

In [ ]:
# !!! not for execution !!! - Begin
# How to specify hyper parameter during auto tuning 

# hyperparameter_ranges_linear = {
#    'alpha': ContinuousParameter(0.01, 10, scaling_type="Linear"),
#    'lambda': ContinuousParameter(0.01, 10, scaling_type="Linear"),
#.   'num_round' : ContinuousParameter(1, 200, scaling_type="Linear")
# }

# tuner_linear = HyperparameterTuner(
#    xgb,
#    objective_metric_name,
#    hyperparameter_ranges_linear,
#    max_jobs=20,
#    max_parallel_jobs=10,
#    strategy='Random'
# )

# tuner_linear.fit({'train': s3_input_train, 'validation': s3_input_validation}, include_cls_metadata=False)

# !!! not for execution !!! - End

### Specify Training Data Location and Optionally, Validation Data Location

In [ ]:
# content type can be libsvm or csv for XGBoost
training_input_config = sagemaker.session.s3_input(s3_data=s3_training_file_location,content_type="csv")
validation_input_config = sagemaker.session.s3_input(s3_data=s3_validation_file_location,content_type="csv")

In [ ]:
## spk ##

train_data      = s3_training_file_location
validation_data = s3_validation_file_location

train_channel = sagemaker.session.s3_input(s3_data=train_data, content_type='text/csv')
valid_channel = sagemaker.session.s3_input(s3_data=validation_data, content_type='text/csv')

data_channels = {'train': train_channel, 'validation': valid_channel}

In [ ]:
print(training_input_config.config)
print(train_channel.config)

In [ ]:
## spk ##

xgb_model.fit(inputs=data_channels,  logs=True)

In [ ]:
xgb_model.fit({'train':train_channel, 'validation':valid_channel})

In [ ]:
print(data_channels)

In [ ]:
print(training_input_config.config)
print(validation_input_config.config)

### Train the model

In [ ]:
# XGBoost supports "train", "validation" channels
# Reference: Supported channels by algorithm
#   https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-algo-docker-registry-paths.html
estimator.fit({'train':training_input_config, 'validation':validation_input_config})
#xgb_model.fit({'train':training_input_config, 'validation':validation_input_config})

## Deploy Model

In [ ]:
# Ref: http://sagemaker.readthedocs.io/en/latest/estimators.html
predictor = estimator.deploy(initial_instance_count=1,
                             instance_type='ml.m4.xlarge',
                             endpoint_name = 'xgboost-biketrain-ver1')

## Run Predictions

In [ ]:
from sagemaker.predictor import csv_serializer, json_deserializer

predictor.content_type = 'text/csv'
predictor.serializer = csv_serializer
predictor.deserializer = None

In [ ]:
predictor.predict([[3,0,1,2,28.7,33.335,79,12.998,2011,7,7,3]])

## Summary

1. Ensure Training, Test and Validation data are in S3 Bucket
2. Select Algorithm Container Registry Path - Path varies by region
3. Configure Estimator for training - Specify Algorithm container, instance count, instance type, model output location
4. Specify algorithm specific hyper parameters
5. Train model
6. Deploy model - Specify instance count, instance type and endpoint name
7. Run Predictions